<a href="https://colab.research.google.com/github/igornunespatricio/Python-Course/blob/main/API/BrasilAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BrasilAPI

In [32]:
import requests
from pprint import pprint
import json
import pandas as pd

## Endpoint Brazilian Banks

In [24]:
# api url
url_banco = 'https://brasilapi.com.br/api/banks/v1'
# getting response
response = requests.get(url_banco)
# getting status
status = response.status_code
print(status)
# printing headers
pprint(response.headers)

200
{'Date': 'Wed, 06 Mar 2024 00:18:28 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'access-control-allow-origin': '*', 'Age': '39727', 'Cache-Control': 'max-age=0, public', 'etag': 'W/"9f5e-wA2jiW6dwXGB4YrXsl3AFw9R3Ls"', 'strict-transport-security': 'max-age=63072000', 'x-matched-path': '/api/banks/v1', 'x-vercel-cache': 'HIT', 'x-vercel-id': 'iad1::sfo1::2hsc5-1709684308191-7651b88bcf28', 'CF-Cache-Status': 'DYNAMIC', 'Report-To': '{"endpoints":[{"url":"https:\\/\\/a.nel.cloudflare.com\\/report\\/v3?s=OETKya6TCrF14%2BQeR6RtvexfqOWDO4fa0rWLMEE%2F8mn2S6Gbfuodoo%2FAIJd%2Bf2XbJg5lTCGGqikL%2Bg9%2BGzZ1DJCsg94USeR3cihTYAFXP2OYr%2FcxUmM7oJdAlGxctmcwbhyG"}],"group":"cf-nel","max_age":604800}', 'NEL': '{"success_fraction":0,"report_to":"cf-nel","max_age":604800}', 'Server': 'cloudflare', 'CF-RAY': '85fe2b2e1ab87bae-ATL', 'Content-Encoding': 'gzip', 'alt-svc': 'h3=":443"; ma=86400'}


In [25]:
# get answer in python list
content = response.json()
pprint(content[:5])

[{'code': 1,
  'fullName': 'Banco do Brasil S.A.',
  'ispb': '00000000',
  'name': 'BCO DO BRASIL S.A.'},
 {'code': 70,
  'fullName': 'BRB - BANCO DE BRASILIA S.A.',
  'ispb': '00000208',
  'name': 'BRB - BCO DE BRASILIA S.A.'},
 {'code': None,
  'fullName': 'Banco Central do Brasil - Selic',
  'ispb': '00038121',
  'name': 'Selic'},
 {'code': None,
  'fullName': 'Banco Central do Brasil',
  'ispb': '00038166',
  'name': 'Bacen'},
 {'code': 539,
  'fullName': 'SANTINVEST S.A. - CREDITO, FINANCIAMENTO E INVESTIMENTOS',
  'ispb': '00122327',
  'name': 'SANTINVEST S.A. - CFI'}]


In [29]:
# write to json file
with open('bank.json', 'w') as file:
    json.dump(content, file)

In [67]:
# load json file in a python list
with open('bank.json', 'r') as file:
    bank_file = json.load(file)
pprint(bank_file[:5])

[{'code': 1,
  'fullName': 'Banco do Brasil S.A.',
  'ispb': '00000000',
  'name': 'BCO DO BRASIL S.A.'},
 {'code': 70,
  'fullName': 'BRB - BANCO DE BRASILIA S.A.',
  'ispb': '00000208',
  'name': 'BRB - BCO DE BRASILIA S.A.'},
 {'code': None,
  'fullName': 'Banco Central do Brasil - Selic',
  'ispb': '00038121',
  'name': 'Selic'},
 {'code': None,
  'fullName': 'Banco Central do Brasil',
  'ispb': '00038166',
  'name': 'Bacen'},
 {'code': 539,
  'fullName': 'SANTINVEST S.A. - CREDITO, FINANCIAMENTO E INVESTIMENTOS',
  'ispb': '00122327',
  'name': 'SANTINVEST S.A. - CFI'}]


In [68]:
# create a pandas dataframe with json response
df_banks = pd.DataFrame(bank_file)
df_banks.head()

,ispb,name,code,fullName
0,00000000,BCO DO BRASIL S.A.,1.0,Banco do Brasil S.A.
1,00000208,BRB - BCO DE BRASILIA S.A.,70.0,BRB - BANCO DE BRASILIA S.A.
2,00038121,Selic,NaN,Banco Central do Brasil - Selic
3,00038166,Bacen,NaN,Banco Central do Brasil
4,00122327,SANTINVEST S.A. - CFI,539.0,"SANTINVEST S.A. - CREDITO, FINANCIAMENTO E INV..."


In [69]:
# get info from pandas dataframe
df_banks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 359 entries, 0 to 358
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ispb      359 non-null    object 
 1   name      357 non-null    object 
 2   code      348 non-null    float64
 3   fullName  357 non-null    object 
dtypes: float64(1), object(3)
memory usage: 11.3+ KB


In [70]:
# mapping where ispb is empty string
empty_ispb_map = df_banks.loc[:,'ispb'] == ''

In [71]:
# remove empty string ispb mapping
df_banks = df_banks.loc[~empty_ispb_map,:]

In [72]:
# banks where code is null: not really banks, but institutions that controls banks, we can remove them
print(df_banks[df_banks['code'].isnull()]['fullName'])
df_banks = df_banks.dropna(subset=['code'])

2           Banco Central do Brasil - Selic
3                   Banco Central do Brasil
10     Secretaria do Tesouro Nacional - STN
40                                  CIP S.A
58                                  CIP S.A
184                    Sistema do Balcão B3
185                                 CIP S.A
285                               Câmara B3
304                     Câmara de Câmbio B3
Name: fullName, dtype: object


In [77]:
# transform code from float to integer
df_banks.code = df_banks.code.astype('int')

In [78]:
# new info after manipulation
df_banks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 348 entries, 0 to 356
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ispb      348 non-null    object
 1   name      348 non-null    object
 2   code      348 non-null    int64 
 3   fullName  348 non-null    object
dtypes: int64(1), object(3)
memory usage: 13.6+ KB


In [80]:
# creating a csv with the transformed data
df_banks.to_csv('bank.csv', index=False)

## Endpoint Brazilian Bank - Specific Bank Information

In [81]:
# url to get specific bank info, code is the unique code of the bank
code = 1
url_specific_bank_info = f'https://brasilapi.com.br/api/banks/v1/{code}'
# getting response
specific_bank_response = requests.get(url_specific_bank_info)
# getting status
status = specific_bank_response.status_code
print(status)
# printing headers
pprint(specific_bank_response.headers)

200
{'Date': 'Wed, 06 Mar 2024 00:53:15 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'access-control-allow-origin': '*', 'Age': '41967', 'Cache-Control': 'max-age=0, public', 'etag': 'W/"5a-huZMZU3M0Pa56jIff8TBob9LjRM"', 'strict-transport-security': 'max-age=63072000', 'x-matched-path': '/api/banks/v1/[code]', 'x-vercel-cache': 'HIT', 'x-vercel-id': 'iad1::sfo1::cvk7f-1709686395971-ba7065c28b3f', 'CF-Cache-Status': 'DYNAMIC', 'Report-To': '{"endpoints":[{"url":"https:\\/\\/a.nel.cloudflare.com\\/report\\/v3?s=pRqZHp6uAhuI1OACHF5N2rxMhn0XP7sBt5lyjTo37EC%2FRxzNs0WLxdg3rOnFFOQeDGjiaTZBlkZr9sKIR9hmRyyc6qCD1zAThF2Ncegcznmz80zybeUdiwvnETNc1KSjplnb"}],"group":"cf-nel","max_age":604800}', 'NEL': '{"success_fraction":0,"report_to":"cf-nel","max_age":604800}', 'Server': 'cloudflare', 'CF-RAY': '85fe5e267d3c672e-ATL', 'Content-Encoding': 'gzip', 'alt-svc': 'h3=":443"; ma=86400'}


In [86]:
# json from the response
content = specific_bank_response.json()
pprint(content)

{'code': 1,
 'fullName': 'Banco do Brasil S.A.',
 'ispb': '00000000',
 'name': 'BCO DO BRASIL S.A.'}


## Endpoint DDD data

In [88]:
# url ddd
ddd = 21
url_ddd = f'https://brasilapi.com.br/api/ddd/v1/{ddd}'
response_ddd = requests.get(url_ddd)
print('status: ',response_ddd.status_code)
print(response_ddd.headers)

status:  200
{'Date': 'Wed, 06 Mar 2024 01:08:56 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'access-control-allow-origin': '*', 'Age': '204', 'Cache-Control': 'max-age=0, public', 'etag': 'W/"154-5oW5jD86feVFDMSU00rUYreLC9g"', 'strict-transport-security': 'max-age=63072000', 'x-matched-path': '/api/ddd/v1/[ddd]', 'x-vercel-cache': 'HIT', 'x-vercel-id': 'iad1::sfo1::rnp4m-1709687336518-e245a4cec321', 'CF-Cache-Status': 'DYNAMIC', 'Report-To': '{"endpoints":[{"url":"https:\\/\\/a.nel.cloudflare.com\\/report\\/v3?s=n69sNPyqXVjIn2UhoMH%2BXf%2B0Y%2B47s4fuKz0HIaCBJn56jZ8fu703NGvcZV7uV40zn5xkwcI2xNdE%2BxVghxIedrXFNOQYM8VumQd1ruC491Y%2B97rQjWjmqHca5aoFITLahlPF"}],"group":"cf-nel","max_age":604800}', 'NEL': '{"success_fraction":0,"report_to":"cf-nel","max_age":604800}', 'Server': 'cloudflare', 'CF-RAY': '85fe751ceb04b0ed-ATL', 'Content-Encoding': 'gzip', 'alt-svc': 'h3=":443"; ma=86400'}


In [92]:
# printing the response from the request
pprint(response_ddd.json())

{'cities': ['TERESÓPOLIS',
            'TANGUÁ',
            'SEROPÉDICA',
            'SÃO JOÃO DE MERITI',
            'SÃO GONÇALO',
            'RIO DE JANEIRO',
            'RIO BONITO',
            'QUEIMADOS',
            'PARACAMBI',
            'NOVA IGUAÇU',
            'NITERÓI',
            'NILÓPOLIS',
            'MESQUITA',
            'MARICÁ',
            'MANGARATIBA',
            'MAGÉ',
            'JAPERI',
            'ITAGUAÍ',
            'ITABORAÍ',
            'GUAPIMIRIM',
            'DUQUE DE CAXIAS',
            'CACHOEIRAS DE MACACU',
            'BELFORD ROXO'],
 'state': 'RJ'}


In [95]:
# create a dataframe and add a column called ddd with value 21
df_ddd = pd.DataFrame(response_ddd.json()).assign(ddd=21)
df_ddd

,state,cities,ddd
0,RJ,TERESÓPOLIS,21
1,RJ,TANGUÁ,21
2,RJ,SEROPÉDICA,21
3,RJ,SÃO JOÃO DE MERITI,21
4,RJ,SÃO GONÇALO,21
5,RJ,RIO DE JANEIRO,21
6,RJ,RIO BONITO,21
7,RJ,QUEIMADOS,21
8,RJ,PARACAMBI,21
9,RJ,NOVA IGUAÇU,21


## Requesting a range of DDDs

In [104]:
# api ddd url
url_ddd = 'https://brasilapi.com.br/api/ddd/v1/'
# get a range of ddds
ddd_list = list(range(20, 31))
# empty dataframe to insert rows from the response
df_ddd = pd.DataFrame()
# loop each ddd in the range
for ddd in ddd_list:
    # request ddd info
    temp_resp = requests.get(''.join([url_ddd, str(ddd)]))
    # print ddd status code
    print(f'DDD: {ddd} - Status code: {temp_resp.status_code}')
    # if status is 200 (successful), concatenate to the end of the dataframe
    if temp_resp.status_code == 200:
        df_ddd = pd.concat(
            [df_ddd, pd.DataFrame(temp_resp.json()).assign(ddd=ddd)]
        )

DDD: 20 - Status code: 404
DDD: 21 - Status code: 200
DDD: 22 - Status code: 200
DDD: 23 - Status code: 404
DDD: 24 - Status code: 200
DDD: 25 - Status code: 404
DDD: 26 - Status code: 404
DDD: 27 - Status code: 200
DDD: 28 - Status code: 200
DDD: 29 - Status code: 404
DDD: 30 - Status code: 404


In [109]:
# reset index of the dataframe
df_ddd.reset_index(drop=True, inplace=True)

In [110]:
df_ddd

,state,cities,ddd
0,RJ,TERESÓPOLIS,21
1,RJ,TANGUÁ,21
2,RJ,SEROPÉDICA,21
3,RJ,SÃO JOÃO DE MERITI,21
4,RJ,SÃO GONÇALO,21
...,...,...,...
165,ES,BOM JESUS DO NORTE,28
166,ES,ATILIO VIVACQUA,28
167,ES,APIACÁ,28
168,ES,ANCHIETA,28


In [111]:
df_ddd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170 entries, 0 to 169
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   state   170 non-null    object
 1   cities  170 non-null    object
 2   ddd     170 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 4.1+ KB


# Requesting Municipalities

In [113]:
fuSymbol = 'RJ'
url_municipality = f'https://brasilapi.com.br/api/ibge/municipios/v1/{fuSymbol}?providers=dados-abertos-br,gov,wikipedia'
response_mun = requests.get(url_municipality)
print(response_mun.status_code)

200


In [115]:
pprint(response_mun.json()[:5])

[{'codigo_ibge': '3300100', 'nome': 'ANGRA DOS REIS'},
 {'codigo_ibge': '3300159', 'nome': 'APERIBÉ'},
 {'codigo_ibge': '3300209', 'nome': 'ARARUAMA'},
 {'codigo_ibge': '3300225', 'nome': 'AREAL'},
 {'codigo_ibge': '3300233', 'nome': 'ARMAÇÃO DOS BÚZIOS'}]


In [117]:
df_mun = pd.DataFrame(response_mun.json())

In [118]:
df_mun.head()

,nome,codigo_ibge
0,ANGRA DOS REIS,3300100
1,APERIBÉ,3300159
2,ARARUAMA,3300209
3,AREAL,3300225
4,ARMAÇÃO DOS BÚZIOS,3300233


In [119]:
df_mun.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92 entries, 0 to 91
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   nome         92 non-null     object
 1   codigo_ibge  92 non-null     object
dtypes: object(2)
memory usage: 1.6+ KB
